**🔧 Setup Required**: Before running this notebook, please follow the [setup instructions](../../README.md#setup-instructions) to configure your environment and API keys.

# Multimodal RAG Pipeline with Vision LLM

This notebook demonstrates how to build a complete multimodal RAG (Retrieval-Augmented Generation) pipeline.

## Pipeline Overview

We'll build a RAG pipeline that:

1. **Retrieves** documents based on text embeddings (using Approach 2 indexing)
2. **Routes** documents by type (images vs PDFs)
3. **Converts** image documents back to image format
4. **Generates** answers using both text and images with a vision LLM

## Why This Approach?

- **Text embeddings for efficient retrieval**: Fast and accurate
- **Original images for accurate visual reasoning**: LLM sees actual images
- **Best of both worlds**: Speed and accuracy

## Key Components

- **Query Embedder**: Converts user questions to vectors
- **Retriever**: Finds relevant documents from our store
- **Document Type Router**: Separates images from text
- **Document to Image Converter**: Prepares images for the LLM
- **Vision LLM**: Generates answers using both modalities

## Setup

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

notebook_dir = os.path.dirname(os.path.abspath("4_multimodal_rag_vision_llm.ipynb")) if os.path.exists("4_multimodal_rag_vision_llm.ipynb") else os.getcwd()

## Step 1: Build the Indexing Pipeline (Approach 2)

First, we need to index our documents using Approach 2 (LLM content extraction).
This creates text embeddings for efficient retrieval.

In [2]:
import urllib.request
import ssl

# Create SSL context
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

def download_file(url, filename):
    """Download file with SSL context"""
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    with urllib.request.urlopen(req, context=ssl_context) as response:
        with open(filename, 'wb') as out_file:
            out_file.write(response.read())

# Create directories
os.makedirs("images", exist_ok=True)
os.makedirs("data_for_indexing", exist_ok=True)

# Download images
download_file(
    "https://upload.wikimedia.org/wikipedia/commons/2/26/Pink_Lady_Apple_%284107712628%29.jpg",
    "images/apple.jpg"
)

# Download PDF
download_file(
    "https://arxiv.org/pdf/1706.03762",
    "data_for_indexing/attention_is_all_you_need.pdf"
)

print("Data downloaded successfully!")

Data downloaded successfully!


In [3]:
from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.converters.image import ImageFileToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.extractors.image import LLMDocumentContentExtractor
from haystack.components.joiners import DocumentJoiner
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack.components.routers.file_type_router import FileTypeRouter
from haystack.components.writers.document_writer import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack import component
from haystack.dataclasses import Document
from typing import List
from haystack.utils import Secret

@component
class ImagePathFixer:
    """Fixes the file paths in image documents to be absolute paths."""
    
    @component.output_types(documents=List[Document])
    def run(self, documents: List[Document]) -> dict:
        for doc in documents:
            if "file_path" in doc.meta:
                file_path = doc.meta["file_path"]
                if os.path.basename(file_path) == file_path:
                    notebook_dir = os.path.dirname(os.path.abspath("4_multimodal_rag_vision_llm.ipynb")) if os.path.exists("4_multimodal_rag_vision_llm.ipynb") else os.getcwd()
                    doc.meta["file_path"] = os.path.join(notebook_dir, "images", file_path)
        return {"documents": documents}

# Create document store
doc_store_text = InMemoryDocumentStore(embedding_similarity_function="cosine")

llm_generator = OpenAIChatGenerator(model="gpt-4o-mini",
                                    api_key=Secret.from_env_var("OPENAI_API_KEY"))

# optional - if you want to use open source model instead of OpenAI
# from haystack_integrations.components.generators.ollama import OllamaGenerator, OllamaChatGenerator
# llm_generator = OllamaChatGenerator(model="mistral-nemo:12b",
#                             generation_kwargs={
#                               "num_predict": 100,
#                               "temperature": 0.9,
#                             })

# Build indexing pipeline
indexing_pipe = Pipeline()
indexing_pipe.add_component("file_type_router", FileTypeRouter(mime_types=["application/pdf", "image/jpeg"]))
indexing_pipe.add_component("pdf_converter", PyPDFToDocument())
indexing_pipe.add_component("pdf_splitter", DocumentSplitter(split_by="page", split_length=1))
indexing_pipe.add_component("image_converter", ImageFileToDocument())
indexing_pipe.add_component("image_path_fixer", ImagePathFixer())
indexing_pipe.add_component(
    "llm_content_extractor",
    LLMDocumentContentExtractor(
        chat_generator=llm_generator,
        max_workers=1
    )
)
indexing_pipe.add_component("doc_embedder", SentenceTransformersDocumentEmbedder(
    model="mixedbread-ai/mxbai-embed-large-v1",
    progress_bar=False
))
indexing_pipe.add_component("final_doc_joiner", DocumentJoiner(sort_by_score=False))
indexing_pipe.add_component("document_writer", DocumentWriter(doc_store_text))

# Connect components
indexing_pipe.connect("file_type_router.application/pdf", "pdf_converter.sources")
indexing_pipe.connect("pdf_converter.documents", "pdf_splitter.documents")
indexing_pipe.connect("pdf_splitter.documents", "final_doc_joiner.documents")
indexing_pipe.connect("file_type_router.image/jpeg", "image_converter.sources")
indexing_pipe.connect("image_converter.documents", "image_path_fixer.documents")
indexing_pipe.connect("image_path_fixer.documents", "llm_content_extractor.documents")
indexing_pipe.connect("llm_content_extractor.documents", "final_doc_joiner.documents")
indexing_pipe.connect("final_doc_joiner.documents", "doc_embedder.documents")
indexing_pipe.connect("doc_embedder.documents", "document_writer.documents")

/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚅 Components
  - file_type_router: FileTypeRouter
  - pdf_converter: PyPDFToDocument
  - pdf_splitter: DocumentSplitter
  - image_converter: ImageFileToDocument
  - image_path_fixer: ImagePathFixer
  - llm_content_extractor: LLMDocumentContentExtractor
  - doc_embedder: SentenceTransformersDocumentEmbedder
  - final_doc_joiner: DocumentJoiner
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.application/pdf -> pdf_converter.sources (list[Union[str, Path, ByteStream]])
  - file_type_router.image/jpeg -> image_converter.sources (list[Union[str, Path, ByteStream]])
  - pdf_converter.documents -> pdf_splitter.documents (list[Document])
  - pdf_splitter.documents -> final_doc_joiner.documents (list[Document])
  - image_converter.documents -> image_path_fixer.documents (list[Document])
  - image_path_fixer.documents -> llm_content_extractor.documents (List[Document])
  - llm_content_extractor.documents -> final_doc_joiner.documents (list[Document])
  - doc_embedder.docu

## Run the Indexing Pipeline

In [4]:
# Index the documents
indexing_result = indexing_pipe.run(
    data={"file_type_router": {"sources": [
        os.path.join(notebook_dir, "data_for_indexing/attention_is_all_you_need.pdf"),
        os.path.join(notebook_dir, "images/apple.jpg")
    ]}}
)

indexed_documents = doc_store_text.filter_documents()
print(f"Indexed {len(indexed_documents)} documents")

Indexed 16 documents


## Step 2: Build the RAG Query Pipeline

Now we'll build the query pipeline that retrieves and generates answers.

### Pipeline Flow:

1. **Query** → Text Embedder → Vector
2. **Vector** → Retriever → Relevant Documents
3. **Documents** → Document Type Router → Separate images and text
4. **Image Documents** → Document to Image Converter → Image Content
5. **Text + Images** → Prompt Builder → Formatted Prompt
6. **Prompt** → Vision LLM → Answer

In [5]:
from haystack.components.embedders.sentence_transformers_text_embedder import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory.embedding_retriever import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.components.converters.image import DocumentToImageContent
from haystack.components.routers import DocumentTypeRouter
from haystack.utils import Secret

# Initialize components
query_text_embedder = SentenceTransformersTextEmbedder(
    model="mixedbread-ai/mxbai-embed-large-v1",
    progress_bar=False
)
retriever = InMemoryEmbeddingRetriever(document_store=doc_store_text, top_k=3)
doc_type_router = DocumentTypeRouter(
    file_path_meta_field="file_path",
    mime_types=["image/jpeg", "application/pdf"]
)
doc_to_image = DocumentToImageContent(detail="auto")

# Chat prompt builder with multimodal template
chat_prompt_builder = ChatPromptBuilder(
    required_variables=["question"],
    template="""{% message role="system" %}
You are a friendly assistant that answers questions based on provided documents and images.
{% endmessage %}

{%- message role="user" -%}
Only provide an answer to the question using the images and text passages provided.

These are the text-only documents:
{%- if documents|length > 0 %}
{%- for doc in documents %}
Text Document [{{ loop.index }}] :
{{ doc.content }}
{% endfor -%}
{%- else %}
No relevant text documents were found.
{% endif %}
End of text documents.

Question: {{ question }}
Answer:

Images:
{%- if image_contents|length > 0 %}
{%- for img in image_contents -%}
  {{ img | templatize_part }}
{%- endfor -%}
{% endif %}
{%- endmessage -%}
"""
)

llm_generator = OpenAIChatGenerator(model="gpt-4o-mini", api_key=Secret.from_env_var("OPENAI_API_KEY"))

# optional - if you want to use open source model instead of OpenAI
# from haystack_integrations.components.generators.ollama import OllamaGenerator, OllamaChatGenerator
# llm_generator = OllamaChatGenerator(model="mistral-nemo:12b",
#                             generation_kwargs={
#                               "num_predict": 100,
#                               "temperature": 0.9,
#                             })

# LLM content extractor for images
llm_content_extractor = LLMDocumentContentExtractor(
    chat_generator= llm_generator,
    max_workers=1
)

print("Query components initialized!")

Query components initialized!


## Build the Query Pipeline

In [ ]:
# Build the query pipeline
rag_pipe = Pipeline()
rag_pipe.add_component("text_embedder", query_text_embedder)
rag_pipe.add_component("retriever", retriever)
rag_pipe.add_component("doc_type_router", doc_type_router)
rag_pipe.add_component("doc_to_image", doc_to_image)
rag_pipe.add_component("chat_prompt_builder", chat_prompt_builder)
rag_pipe.add_component("llm", llm_generator)

# Connect components
rag_pipe.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipe.connect("retriever.documents", "doc_type_router.documents")
rag_pipe.connect("doc_type_router.image/jpeg", "doc_to_image.documents")
rag_pipe.connect("doc_to_image.image_contents", "chat_prompt_builder.image_contents")
rag_pipe.connect("doc_type_router.application/pdf", "chat_prompt_builder.documents")
rag_pipe.connect("chat_prompt_builder.prompt", "llm.messages")

RAG pipeline built!


## Visualize the RAG Pipeline

In [7]:
# Visualize the RAG pipeline
rag_pipe.draw(path="images/multimodal_rag.png")

![](./images/multimodal_rag.png)

![](./images/multimodal_rag.png)

## Query the RAG Pipeline

Now let's ask questions about our indexed content!

### Query 1: Ask about the Image

In [8]:
# Query about the image
query = "What is the color of the background of the image with an apple in it?"
result = rag_pipe.run(
    data={
        "text_embedder": {"text": query},
        "chat_prompt_builder": {"question": query}
    }
)

print(f"Question: {query}")
print(f"\nAnswer: {result['llm']['replies'][0].text}")

Question: What is the color of the background of the image with an apple in it?

Answer: The background of the image with the apple is a light brown color.


### Query 2: Ask about the PDF Document

In [9]:
# Query about the PDF document
query = "What is attention in the transformers architecture?"
result = rag_pipe.run(
    data={
        "text_embedder": {"text": query},
        "chat_prompt_builder": {"question": query}
    }
)

print(f"Question: {query}")
print(f"\nAnswer: {result['llm']['replies'][0].text}")

Question: What is attention in the transformers architecture?

Answer: Attention in the transformers architecture is a mechanism that maps a query and a set of key-value pairs to an output, where the output is computed as a weighted sum of the values. The weights are determined by a compatibility function of the query with the corresponding keys. Specifically, the attention function is calculated using the formula:

Attention(Q, K, V) = softmax(QKT / √dk) * V 

This allows the model to focus on relevant parts of the input sequence, thereby improving performance on tasks such as sequence transduction. Multi-head attention extends this by allowing the model to jointly attend to information from different representation subspaces at different positions, enhancing the richness of the information processed.


### Query 3: Your Custom Question

Try asking your own questions!

In [10]:
# Try your own query
query = "Describe the apple image in detail"
result = rag_pipe.run(
    data={
        "text_embedder": {"text": query},
        "chat_prompt_builder": {"question": query}
    }
)

print(f"Question: {query}")
print(f"\nAnswer: {result['llm']['replies'][0].text}")

Question: Describe the apple image in detail

Answer: I'm unable to describe the apple image in detail.


## Inspect Retrieved Documents

Let's see which documents were retrieved for a query:

In [15]:
# Run pipeline to get intermediate results
# We'll need to enable include_outputs_from to see intermediate results
query = "What is the apple?"
result = rag_pipe.run(
    data={
        "text_embedder": {"text": query},
        "chat_prompt_builder": {"question": query}
    },
    include_outputs_from={"retriever", "doc_type_router"}
)

# Check retrieved documents
retrieved_docs = result['retriever']['documents']
print(f"Retrieved {len(retrieved_docs)} documents:\n")
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"  File: {doc.meta.get('file_path', 'Unknown')}")
    print(f"  Score: {doc.score:.4f}")
    print(f"  Content preview: {str(doc.content)[:100]}...\n")

Retrieved 3 documents:

Document 1:
  File: /Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/jupyter-notebooks/multimodal/images/apple.jpg
  Score: 0.7125
  Content preview: [img-caption]A red and yellow apple resting on a textured brown surface.[/img-caption]...

Document 2:
  File: attention_is_all_you_need.pdf
  Score: 0.4056
  Content preview: Attention Visualizations
Input-Input Layer5
It
is
in
this
spirit
that
a
majority
of
American
governm...

Document 3:
  File: attention_is_all_you_need.pdf
  Score: 0.3980
  Content preview: Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture us...



## Summary

In this notebook, we built a complete multimodal RAG pipeline:

1. **Indexed documents** using Approach 2 (LLM content extraction)
2. **Built a query pipeline** that retrieves using text embeddings
3. **Converted image documents** back to images for the vision LLM
4. **Generated answers** using both text and images

### Key Features:
- **Efficient retrieval**: Text embeddings for fast search
- **Accurate generation**: Original images for vision LLM
- **Multimodal understanding**: Works with both text and images

### Pipeline Architecture Benefits:
1. **Separation of concerns**: Retrieval and generation are independent
2. **Flexibility**: Can swap out components easily
3. **Scalability**: Can handle large document collections

### Best Practices:
- Use text embeddings for retrieval (faster)
- Use original images for generation (more accurate)
- Adjust `top_k` in retriever based on your needs
- Monitor LLM costs (vision LLMs are more expensive)

### Next Steps:
- Continue to notebook 5 to add audio transcription with Whisper
- Experiment with different vision LLMs (GPT-4 Vision, Claude 3, etc.)
- Try different embedding models
- Add more document types to your index